In [2]:
from USTs import USTs
import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import newton
import pandas_market_calendars as mcal
import pandas as pd
from pandas import Timestamp
import requests
from bs4 import BeautifulSoup

In [13]:
PRICE = 92.25
ISSUE = datetime.datetime(2024, 11, 15)
RATE = 4.500
MATURITY = datetime.datetime(2054, 11, 15)
SETTLEMENT = datetime.datetime(2025, 6, 3)

In [14]:
USTs(None, None).get_coupon_ytm(PRICE, ISSUE, MATURITY, SETTLEMENT, RATE, dirty=False)

5.007342

In [6]:
row = pd.DataFrame([{'End of day': PRICE,
           'Maturity date': MATURITY,
           'Issue date': ISSUE,
           'EOD YTM': YTM,
           'Rate': RATE}])

In [ ]:
USTs(auction_data= None, price_data=None)._get_duration(row=row, settlement_date=SETTLEMENT.date())

In [15]:
import rateslib as rl
bond = rl.FixedRateBond(
    effective=ISSUE,
    termination=MATURITY,
    fixed_rate = RATE,
    spec="us_gb"  # US Government Bond
)
rateslib = bond.ytm(price=PRICE, settlement=SETTLEMENT)
rateslib

5.005743585103647

In [ ]:
all_data = dict()
CUSIPS = ['912810UA4', '912810UC0', '912810UE6']
for date in pd.date_range(start=datetime.date(2025, 6, 1), end=datetime.date(2025, 6, 10)):
    date_price_info = dict()
    payload = {
                "priceDate.month": date.month,
                "priceDate.day": date.day,
                "priceDate.year": date.year,
                "submit": "Show Prices"
            }
    response = requests.post(url="https://www.treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate",
                            data=payload, timeout=30)
    response.raise_for_status()

    output = BeautifulSoup(response.text, "html.parser")
    table = output.find("table", class_="data1")
    header_row = table.find_all("th")
    data = table.find_all("tr")[1:]
    if not data:
        print("No data on this date")
    else:
        headers = [header.get_text(strip=True).capitalize() for header in header_row]
        for tr in data:
            data_row = [td.get_text(strip=True) for td in tr.find_all("td")]
            if data_row[0] in CUSIPS: date_price_info[data_row[0]] = data_row[7]
    all_data[date] = date_price_info
df = pd.DataFrame(data=all_data).transpose().reset_index(drop=False).rename(columns={"index": "Date"}).dropna(axis=0)

No data on this date
No data on this date
No data on this date
